In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import anndata as ad
import scanpy.external as sce
from sklearn import preprocessing
import pickle5 as pickle
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
from sklearn import preprocessing
import sklearn
from sklearn.metrics import accuracy_score
import seaborn as sns
from statannotations.Annotator import Annotator
from scipy.stats import pearsonr

from utils import *


eps=1e-100


In [ ]:
from pathlib import Path
# Function to update sums and counts
def update_sums_counts(points, values, sums, counts):
    for point, value in zip(points, values):
        grid_x = min(int((point[0] - min_x) / cell_size_x), grid_size_x - 1)
        grid_y = min(int((point[1] - min_y) / cell_size_y), grid_size_y - 1)
        sums[grid_x, grid_y] += value
        counts[grid_x, grid_y] += 1
np.random.seed(0)

In [ ]:
ad_gene_embedding=sc.read_h5ad("../source_data/ad_embed.h5ad")
ct_embed = sc.read_h5ad('../source_data/cell_type/transfer_celltype_label.h5ad')


In [ ]:
focus_slice=112

In [ ]:
# for focus_slice  in range(45,404):
locate_batch=ct_embed[ct_embed.obs['ap_order']==focus_slice].obs['batch'][0]
locate_batch_1=ct_embed[ct_embed.obs['ap_order']==focus_slice+5].obs['batch'][0]

a_name = ct_embed[ct_embed.obs['batch']==locate_batch].obs['name'][0]
b_name = ct_embed[ct_embed.obs['batch']==locate_batch_1].obs['name'][0]

print(focus_slice, a_name, b_name)

locint = int(locate_batch.split('sample')[1])

In [ ]:

adata_raw_o=sc.read_h5ad('../source_data/STARmap_well03.h5ad')

# get imputed slice
imputed = ct_embed[ct_embed.obs['batch']==locate_batch]
ad_imputed_o = ad.AnnData(X=imputed.X@ad_gene_embedding.X.T)
ad_imputed_o.var.index = ad_gene_embedding.obs.index
sc.pp.scale(ad_imputed_o, zero_center=False, max_value=10)


# subset to global coord
ad_imputed_o.obs.index = ct_embed[ct_embed.obs['batch']==locate_batch].obs.index
adata_raw_o.obs.index = ct_embed[ct_embed.obs['batch']==locate_batch].obs.index

ad_imputed_o.obs['global_x'] = ct_embed[ct_embed.obs['batch']==locate_batch].obs['global_x']
ad_imputed_o.obs['global_y'] = ct_embed[ct_embed.obs['batch']==locate_batch].obs['global_y']
ad_imputed_o.obs['global_z'] = ct_embed[ct_embed.obs['batch']==locate_batch].obs['global_z']

adata_raw_o.obs['global_x'] = ct_embed[ct_embed.obs['batch']==locate_batch].obs['global_x']
adata_raw_o.obs['global_y'] = ct_embed[ct_embed.obs['batch']==locate_batch].obs['global_y']
adata_raw_o.obs['global_z'] = ct_embed[ct_embed.obs['batch']==locate_batch].obs['global_z']

ad_imputed = ad_imputed_o[~ad_imputed_o.obs['global_x'].isna()]
adata_raw = adata_raw_o[~adata_raw_o.obs['global_x'].isna()]

In [ ]:
# get near slice
focus_slice = focus_slice+3
locint = int(locate_batch_1.split('sample')[1])

adata_raw_near_o=sc.read_h5ad('../source_data/slideseq_Puck24.h5ad')
adata_raw_near_o.obs.index = ct_embed[ct_embed.obs['batch']==locate_batch_1].obs.index


adata_raw_near_o.obs['global_x'] = ct_embed[ct_embed.obs['batch']==locate_batch_1].obs['global_x']
adata_raw_near_o.obs['global_y'] = ct_embed[ct_embed.obs['batch']==locate_batch_1].obs['global_y']
adata_raw_near_o.obs['global_z'] = ct_embed[ct_embed.obs['batch']==locate_batch_1].obs['global_z']

adata_raw_near = adata_raw_near_o[~adata_raw_near_o.obs['global_x'].isna()]


## prepare to grid
array_1 = np.array(adata_raw_near.obs[['global_y','global_z']])  # replace with your data
array_2 = np.array(ad_imputed.obs[['global_y','global_z']])  # replace with your data


all_points = np.vstack((array_1, array_2))
min_x, max_x = np.min(all_points[:, 0]), np.max(all_points[:, 0])
min_y, max_y = np.min(all_points[:, 1]), np.max(all_points[:, 1])

grid_size_x, grid_size_y = int((max_x-min_x)/(max_y-min_y)*int((max_y-min_y)/10)), int((max_y-min_y)/10)

cell_size_x = (max_x - min_x) / grid_size_x
cell_size_y = (max_y - min_y) / grid_size_y

uncomment to compute pearson correlation

In [ ]:
# # new gene imputed
# dic_all={'gene':[],'corr':[]}
# for i in tqdm(adata_raw_near.var.index):
#     if i not in adata_raw.var.index:
#         select_gene = i

#         normalized_data = ad_imputed.X[:,np.where(ad_imputed.var.index==select_gene)[0][0]].reshape(-1, 1).flatten()
#         normalized_data[normalized_data<0]=0

#         normalized_data_1 = adata_raw_near[:,select_gene].X.toarray()

#         value_1 = normalized_data_1
#         value_2 = normalized_data

#         sums_1 = np.zeros((grid_size_x, grid_size_y))
#         counts_1 = np.zeros((grid_size_x, grid_size_y))
#         sums_2 = np.zeros((grid_size_x, grid_size_y))
#         counts_2 = np.zeros((grid_size_x, grid_size_y))

#         update_sums_counts(array_1, value_1, sums_1, counts_1)
#         update_sums_counts(array_2, value_2, sums_2, counts_2)

#         # Calculate mean values for each grid cell
#         means_1 = np.divide(sums_1, counts_1, out=np.zeros_like(sums_1), where=counts_1!=0)
#         means_2 = np.divide(sums_2, counts_2, out=np.zeros_like(sums_2), where=counts_2!=0)

#         # Prepare a list for 2D coordinates and their corresponding mean values
#         coordinates_means_1 = []
#         values_means_1 = []
#         coordinates_means_2 = []
#         values_means_2 = []

#         for i in range(grid_size_x):
#             for j in range(grid_size_y): 
#                     coordinates_means_1.append([i, j])
#                     values_means_1.append(means_1[i, j])
#                     values_means_2.append(means_2[i, j])

#         corr, p_value = pearsonr(values_means_1, values_means_2)
#         dic_all['gene'].append(select_gene)
#         dic_all['corr'].append(corr)

        

# # old gene raw
# dic_all_gt={'gene':[],'corr':[]}
# for i in tqdm(adata_raw_near.var.index):
#     if i in adata_raw.var.index:
#         select_gene = i
#         normalized_data = adata_raw.X[:,np.where(adata_raw.var.index==select_gene)[0][0]].toarray().reshape(-1, 1).flatten()

#         normalized_data_1 = adata_raw_near[:,select_gene].X.toarray()

#         value_1 = normalized_data_1
#         value_2 = normalized_data

#         # Initialize arrays for sums and counts for each array
#         sums_1 = np.zeros((grid_size_x, grid_size_y))
#         counts_1 = np.zeros((grid_size_x, grid_size_y))
#         sums_2 = np.zeros((grid_size_x, grid_size_y))
#         counts_2 = np.zeros((grid_size_x, grid_size_y))

#         # Update sums and counts for each array
#         update_sums_counts(array_1, value_1, sums_1, counts_1)
#         update_sums_counts(array_2, value_2, sums_2, counts_2)

#         # Calculate mean values for each grid cell
#         means_1 = np.divide(sums_1, counts_1, out=np.zeros_like(sums_1), where=counts_1!=0)
#         means_2 = np.divide(sums_2, counts_2, out=np.zeros_like(sums_2), where=counts_2!=0)

#         # Prepare a list for 2D coordinates and their corresponding mean values
#         coordinates_means_1 = []
#         values_means_1 = []
#         coordinates_means_2 = []
#         values_means_2 = []

#         for i in range(grid_size_x):
#             for j in range(grid_size_y): 
#                     # Append the coordinates and the mean value
#                     coordinates_means_1.append([i, j])
#                     values_means_1.append(means_1[i, j])
#                     values_means_2.append(means_2[i, j])

#         corr, p_value = pearsonr(values_means_1, values_means_2)
# #         print(select_gene,corr)
#         dic_all_gt['gene'].append(select_gene)
#         dic_all_gt['corr'].append(corr)




# # old gene imputed
# dic_all_gt_imputed={'gene':[],'corr':[]}
# for i in tqdm(adata_raw_near.var.index):
#     if i in adata_raw.var.index:
#         select_gene = i

#         normalized_data = ad_imputed.X[:,np.where(ad_imputed.var.index==select_gene)[0][0]].reshape(-1, 1).flatten()
#         normalized_data[normalized_data<0]=0

#         normalized_data_1 = adata_raw_near[:,select_gene].X.toarray()

#         array_1 = np.array(adata_raw_near.obs[['global_y','global_z']])  # replace with your data
#         array_2 = np.array(ad_imputed.obs[['global_y','global_z']])  # replace with your data

#         value_1 = normalized_data_1
#         value_2 = normalized_data

#         # Initialize arrays for sums and counts for each array
#         sums_1 = np.zeros((grid_size_x, grid_size_y))
#         counts_1 = np.zeros((grid_size_x, grid_size_y))
#         sums_2 = np.zeros((grid_size_x, grid_size_y))
#         counts_2 = np.zeros((grid_size_x, grid_size_y))

#         # Update sums and counts for each array
#         update_sums_counts(array_1, value_1, sums_1, counts_1)
#         update_sums_counts(array_2, value_2, sums_2, counts_2)

#         # Calculate mean values for each grid cell
#         means_1 = np.divide(sums_1, counts_1, out=np.zeros_like(sums_1), where=counts_1!=0)
#         means_2 = np.divide(sums_2, counts_2, out=np.zeros_like(sums_2), where=counts_2!=0)

#         # Prepare a list for 2D coordinates and their corresponding mean values
#         coordinates_means_1 = []
#         values_means_1 = []
#         coordinates_means_2 = []
#         values_means_2 = []

#         for i in range(grid_size_x):
#             for j in range(grid_size_y): 
#                     # Append the coordinates and the mean value
#                     coordinates_means_1.append([i, j])
#                     values_means_1.append(means_1[i, j])
#                     values_means_2.append(means_2[i, j])

#         corr, p_value = pearsonr(values_means_1, values_means_2)
# #         print(select_gene,corr)
#         dic_all_gt_imputed['gene'].append(select_gene)
#         dic_all_gt_imputed['corr'].append(corr)


        
# dic_plot={'source':[],'gene':[],'corr':[]}

# for i,j in zip(dic_all['gene'],dic_all['corr']):
#     dic_plot['source'].append('new_imputed')
#     dic_plot['gene'].append(i)
#     dic_plot['corr'].append(j)

# for i,j in zip(dic_all_gt_imputed['gene'],dic_all_gt_imputed['corr']):
#     dic_plot['source'].append('overlap_imputed')
#     dic_plot['gene'].append(i)
#     dic_plot['corr'].append(j)
# for i,j in zip(dic_all_gt['gene'],dic_all_gt['corr']):
#     dic_plot['source'].append('overlap_measured')
#     dic_plot['gene'].append(i)
#     dic_plot['corr'].append(j)

# dic_plot=pd.DataFrame(dic_plot)


# savepppth=f'../source_data/result/'
# Path(savepppth).mkdir(parents=True, exist_ok=True)
# dic_plot.to_csv(savepppth+f'/{a_name}_{b_name}_{112}_{117}.csv')


#### violin plot

In [ ]:
savepppth=f'../source_data/result/STARmap_Slideseq_112_117.csv'
pd_all=pd.read_csv(savepppth,index_col=0)

In [ ]:
# with plt.style.context(("seaborn-dark",)):

#     rc = {'figure.figsize':(5,5),
#           'axes.facecolor':'white',
#           'font.size' : 10}
#     plt.rcParams.update(rc)

x_order=['overlap_measured','overlap_imputed','new_imputed']
g= sns.violinplot(x=pd_all["source"], y=pd_all["corr"],fliersize=0,
                 zorder=1,order=x_order)
plt.ylim([-0.2,1])
plt.show()

plt.close()

### plot shared genes

In [ ]:
plot_gene_all = ['TCF4','SOX11','SCN4B','MBP']
for select_gene in plot_gene_all:
    
    normalized_data = ad_imputed.X[:,np.where(ad_imputed.var.index==select_gene)[0][0]].reshape(-1, 1)
    normalized_data = normalized_data.flatten()
    normalized_data[normalized_data<0]=0
    normalized_data_1 = adata_raw_near[:,select_gene].X.toarray()
    normalized_data_raw = adata_raw.X[:,np.where(adata_raw.var.index==select_gene)[0][0]].toarray().reshape(-1, 1)



    x=adata_raw.obs['x']
    y=adata_raw.obs['y']
    coff=(max(x)-min(x))/(max(y)-min(y))

    plt.figure(figsize=(5,5/coff))
    plt.scatter(x,y,s=0.1,
                c=normalized_data_raw,cmap='Purples')
    plt.scatter(x[[normalized_data_raw>0][0].flatten()],y[[normalized_data_raw>0][0].flatten()],s=1,
                c=normalized_data_raw[[normalized_data_raw>0][0].flatten()],cmap='Purples')
    plt.axis('off')
    plt.gca().invert_yaxis()
    plt.title(f'Section 208 {select_gene} Measured expression')
    plt.show()
    plt.close()



    x=adata_raw.obs['x']
    y=adata_raw.obs['y']
    coff=(max(x)-min(x))/(max(y)-min(y))
    plt.figure(figsize=(5,5/coff))
    plt.scatter(x,y,s=0.1,
                c=normalized_data,cmap='Purples')
    plt.scatter(x[[normalized_data>0][0].flatten()],y[[normalized_data>0][0].flatten()],s=1,
                c=normalized_data[[normalized_data>0][0].flatten()],cmap='Purples')
    plt.axis('off')
    plt.gca().invert_yaxis()
    plt.title(f'Section 208 {select_gene} Imputed expression')
    plt.show()
    plt.close()




    x=adata_raw_near.obs['x']
    y=adata_raw_near.obs['y']
    coff=(max(x)-min(x))/(max(y)-min(y))
    plt.figure(figsize=(5,5*coff))
    plt.scatter(y,x,s=1,
                c=normalized_data_1,cmap='Purples')
    plt.scatter(y[[normalized_data_1>0][0].flatten()],x[[normalized_data_1>0][0].flatten()],s=1,
                c=normalized_data_1[[normalized_data_1>0][0].flatten()],cmap='Purples')
    plt.axis('off')
    plt.gca().invert_xaxis()
    plt.title(f'Section 211 {select_gene} Measured expression')
    plt.show()
    plt.close()


### plot distinct genes

In [ ]:
plot_gene_all = ['PDE10A','YBX1','ATP2B1','DLG2']
for select_gene in plot_gene_all:
    
    normalized_data = ad_imputed.X[:,np.where(ad_imputed.var.index==select_gene)[0][0]].reshape(-1, 1)
    normalized_data = normalized_data.flatten()
    normalized_data[normalized_data<0]=0
    normalized_data_1 = adata_raw_near[:,select_gene].X.toarray()

    
    x=adata_raw.obs['x']
    y=adata_raw.obs['y']
    coff=(max(x)-min(x))/(max(y)-min(y))
    plt.figure(figsize=(5,5/coff))
    plt.scatter(x,y,s=0.1,
                c=normalized_data,cmap='Purples')
    plt.scatter(x[[normalized_data>0][0].flatten()],y[[normalized_data>0][0].flatten()],s=1,
                c=normalized_data[[normalized_data>0][0].flatten()],cmap='Purples')
    plt.axis('off')
    plt.gca().invert_yaxis()
    plt.title(f'Section 208 {select_gene} Imputed expression')
    plt.show()
    plt.close()




    x=adata_raw_near.obs['x']
    y=adata_raw_near.obs['y']
    coff=(max(x)-min(x))/(max(y)-min(y))
    plt.figure(figsize=(5,5*coff))
    plt.scatter(y,x,s=1,
                c=normalized_data_1,cmap='Purples')
    plt.scatter(y[[normalized_data_1>0][0].flatten()],x[[normalized_data_1>0][0].flatten()],s=1,
                c=normalized_data_1[[normalized_data_1>0][0].flatten()],cmap='Purples')
    plt.axis('off')
    plt.gca().invert_xaxis()
    plt.title(f'Section 211 {select_gene} Measured expression')
    plt.show()
    plt.close()
